In [8]:
#import libraries
import numpy as np
import random

In [9]:
#create an environment for the game
class GameEnvironment():
    def __init__(self, grid, start_state=(0, 0)):
        self.grid = grid
        self.rows, self.cols = grid.shape
        self.start_state = start_state
        self.num_turns = 0
        
    def get_reward(self): #return reward obtained in current state
        if self.grid[self.state[0]][self.state[1]] == 1:
            return 1
        else:
            return -1
    
    def is_valid(self, state):   # checks if this is a valid state(move)
        if state[0] < 0 or state[0] >= self.rows:
            # outside row bound
            return False
        elif state[1] < 0 or state[1] >= self.cols:
            # outside column bound
            return False
        elif np.isnan(self.grid[state[0]][state[1]]):
            # wall
            return False
        return True
    def get_next_state(self, action):    # update state according to action taken
        if action == "up":
            next_state = (self.state[0]-1, self.state[1])
        elif action == "down":
            next_state = (self.state[0]+1, self.state[1])
        elif action == "right":
            next_state = (self.state[0], self.state[1]+1)
        elif action == "left":
            next_state = (self.state[0], self.state[1]-1)
        else:
            raise ValueError("Invalid action!")
        if self.is_valid(next_state):
            return next_state
        else:
            return self.state
        
    def reset(self):   #reset states
        self.state = self.start_state
        self.num_turns = 0
    
    def is_end(self):     #Returns true if the game has come to an end
        val = self.grid[self.state[0]][self.state[1]]
        return val == 1 or self.num_turns == 100

In [10]:
#creating an Agent
class CreateAgent():
    def __init__(self, game):
        self.game = game
        self.lr = 0.2
        self.explore_rate = 0.1
        self.gamma = 0.95
        self.actions = ["up", "down", "left", "right"]
        
        self.state_values = {}
        for i in range(self.game.rows):
            for j in range(self.game.cols):
                self.state_values[(i, j)] = 0
                
    def get_valid_actions(self):    #Get a list of valid actions from current position
        valid = []
        for action in self.actions:
            state = self.game.get_next_state(action)
            if state != self.game.state:
                valid.append(action)
        return valid
        
    
    def select_action(self):
        valid_actions = self.get_valid_actions()
        # if we don't shuffle, the agent may follow same path over and over again 
        # (in the order) and never reach the global minima
        random.shuffle(valid_actions) 
        if np.random.uniform(0, 1) <= self.explore_rate:
            action = np.random.choice(valid_actions)
        else:
            max_reward = -np.inf
            action = ""
            for a in valid_actions:
                exp_reward = self.state_values[self.game.get_next_state(a)]
                if exp_reward >= max_reward:
                    max_reward = exp_reward
                    action = a
        return action
    
    def play_step(self, max_iter=1000):
        self.game.reset()
        it = 0
        path = []
        path.append(self.game.state)
        while True:
            action = self.select_action()
            self.game.state = self.game.get_next_state(action)
            self.game.num_turns += 1
            path.append(self.game.state)
            
            if self.game.is_end():
                reward = self.game.get_reward()
                for state in reversed(path):
                    self.state_values[state] += self.lr * (reward-self.state_values[state])
                    reward *= self.gamma
                return path
    
    def play(self, ite):
        for _ in range(ite):
            path = self.play_step()
            self.explore_rate *= 0.9
        print('Shortest path found with length:', len(path))
        print(path)

In [13]:
#Testing
N = np.nan
grid = np.array([
    [0, N, 0, 0, N, 0], 
    [0, N, 0, 0, N, 0], 
    [0, 0, 0, 0, 0, 0], 
    [0, 0, N, 0, 1, 0]
])
start_state=(0, 0)
game = GameEnvironment(grid, start_state)
agent = CreateAgent(game)
agent.play(100)

Shortest path found with length: 8
[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4)]
